In [1]:
import h5py
import numpy as np
import pickle
import random
import pdb
from generator_helper_functions import create_user_maxtrix, create_user_ids, latent_user_input_layer

# f = h5py.File('../data/Full_NF.h5', 'r')
# dset = f['dataset_1']
# dset = dset[:50000000]
# print("Data loaded from file")

# dset_indicies = set(range(len(dset)))
# train_num = int(.9 * len(dset))
# print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

# user_ids = create_user_ids(dset)
# train_indicies = random.sample(dset_indicies, train_num)
# validation_indicies = dset_indicies.difference(train_indicies)
# training_data = np.array([dset[i] for i in train_indicies])
# del train_indicies
# validation_data = np.array([dset[i] for i in validation_indicies])
# del validation_indicies
# print("Training and Validation datasets separated")
# #del dset

with open('../data/training_data.pickle', 'rb') as handle:
    training_data = pickle.load(handle)
    
hf = h5py.File('../data/training_data.h5', 'w')
hf.create_dataset('training_data', data=training_data)
del training_data
training_data = hf["training_data"]
del hf

    
with open('../data/validation_data.pickle', 'rb') as handle:
    validation_data = pickle.load(handle)
    
hf = h5py.File('../data/validation_data.h5', 'w')
hf.create_dataset('validation_data', data=validation_data)
del validation_data
validation_data = hf["validation_data"]
del hf



/home/cody/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
f = h5py.File('../data/Full_NF.h5', 'r')
dset = f['dataset_1']
dset = dset[:]

In [ ]:
dset_mean = np.mean(dset[:,2])
del dset

In [ ]:

f = h5py.File('../data/Full_NF.h5', 'r')
dset = f['dataset_1']
dset = dset[:100]
user_ids = create_user_ids(dset)
k = 50

In [ ]:
#train_indices[0]

In [ ]:
#del dset_indicies
#del train_indicies
#del validation_indicies
del dset

In [3]:
user_matrix = create_user_maxtrix(training_data[:], user_ids, k_n=50)

MemoryError: 

In [ ]:
user_matrix[0]

In [ ]:
import pickle

In [ ]:
# with open('../data/user_matrix.pickle', 'wb') as handle:
#     pickle.dump(user_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('../data/training_data.pickle', 'wb') as handle:
#     pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('../data/validation_data.pickle', 'wb') as handle:
#     pickle.dump(validation_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/user_ids.pickle', 'wb') as handle:
    pickle.dump(user_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
del user_matrix
del training_data
del validation_data
del user_ids

In [ ]:
hf = h5py.File('../data/training_data.h5', 'r')
del training_data
training_data = hf["training_data"]
del hf

In [2]:
import pickle
import h5py
with open('../data/user_matrix.pickle', 'rb') as handle:
    user_matrix = pickle.load(handle)
# with open('../data/training_data.pickle', 'rb') as handle:
#     training_data = pickle.load(handle)
# with open('../data/validation_data.pickle', 'rb') as handle:
#     validation_data = pickle.load(handle)
with open('../data/user_ids.pickle', 'rb') as handle:
    user_ids = pickle.load(handle)
# hf = h5py.File('../data/training_data.h5', 'r')
# training_data = hf["training_data"]
# del hf
# hf = h5py.File('../data/validation_data.h5', 'r')
# validation_data = hf["validation_data"]
# del hf

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
transformer = MinMaxScaler(copy=False)
transformer.fit_transform(user_matrix)
user_matrix

In [ ]:
k = 50      
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
#import random
import numpy as np
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import  latent_user_input_layer, Generate_User_Latent_Feedforward

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()


# k dim user vector + number of movies + statistics + month, day, year
input_size = k + 17770

batch_size = 2

model = Sequential()
model.add(Dense(input_size, activation='relu', input_shape=(input_size,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mse',
             optimizer='sgd',
             metrics=['mae', 'acc'])

print("Model Compiled")

In [ ]:

train_gen = Generate_User_Latent_Feedforward(training_data, 
                                batch_size, 
                                (input_size,), 
                                (1,), #label shape
                                latent_user_input_layer, 
                                lambda x: x[2],
                                user_matrix,
                                user_ids
                                )

validation_gen = Generate_User_Latent_Feedforward(validation_data, 
                                batch_size, 
                                (input_size,), 
                                (1,), #label shape
                                latent_user_input_layer, 
                                lambda x: x[2],
                                user_matrix,
                                user_ids
                                )

checkpointer = ModelCheckpoint(filepath='../models/latent_user_mse_K50_regression_v1.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=2,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=100,
                    callbacks=[checkpointer]
                    )

In [ ]:
model.load_weights('../models/latent_user_SGD_MSE_K50_cat_v2_normalized.hdf5')

In [ ]:
validation_data[1]

In [ ]:
from sklearn.metrics import precision_score

In [ ]:

list(map(lambda x: float(model.predict(np.expand_dims(latent_user_input_layer(x, user_matrix, user_ids, True), axis=0)) * 4 + 1), 
             validation_data[0:30]))




In [ ]:
validation_data[0:30, 2]

In [ ]:
user_ids = {user:i for i, user in enumerate(user_ids)}

In [ ]:
user_rating_matrix

In [ ]:
user_rating_matrix